## Implicit-Explicit finite difference schemes for the advection-diffusion equation
[Connor P. Duffin](https://connorpduffin.github.io), *The Unversity of Western Australia*.

This is a brief description of a method of implementing an *implicit-explicit* (IMEX) finite difference scheme for the advection-diffusion equation. IMEX methods are used when we have two parts of a PDE that are individually suited to difference forms of finite differencing. In learning these methods, I found that there were not many well-documented implementations available for practitioners on the internet. Whilst not claiming to be well-documented, I hope this is able to help someone who is trying to learn this stuff for the first time. This document proceeds as follows. We start by presenting the details of IMEX schemes, before moving onto the advection-diffusion equation. We then derive the finite difference scheme of interest, and implement it in `Python`. We conclude with a presentation of the resulting output.

### IMEX schemes
Consider a (time-dependent) PDE of the form:
\begin{equation*}
  u_t = f(u) + g(u)
\end{equation*}
where $f$ and $g$ may be differential operators, but we will just think of them as functions (for now). This allows us to borrow techniques from the ODE solving literature. Assume that $f$ may be nonlinear, so we would like to avoid solving any nonlinear equations. We will use an *explicit* routine, therefore, for these terms. We will also assume that $g$ is stiff, and integrating this part of the equation using typical explicit schemes is not very stable. For these terms, we will use an *implicit* routine.

#### The explicit scheme
First, let's look at the system in terms of just $f$. Suppose that we had a equation to solve, explicitly, that looks like
\begin{equation*}
  u_t = f(u).
\end{equation*}
Remember that in order to solve an equation explicitly, we will solve the equation only in terms of previous values (that we know). The explicit solution method that we are interested in will actually be a method of solving ODE's, called the [Adams-Bashfourth method](https://en.wikipedia.org/wiki/Linear_multistep_method#Adams%E2%80%93Bashforth_methods). The Adams-Bashfourth method belongs to a family of methods that are available to solve ODE's using more than a single timestep. These general methods are of the form:
\begin{equation*}
  y_{n + 1} = y_n + \Delta t \left(a_1 y_{n - 1} + a_2 y_{n - 2} + a_3 y_{n - 3} + \cdots\right),
\end{equation*}
for some coefficients $a_k$, $k = 1, 2, \ldots$. The second-order Adams-Bashfourth method is of the form:
\begin{equation*}
  y_{n + 1} = y_n + \Delta t \left(\frac{3}{2} f(y_{n - 1}) - \frac{1}{2} f(y_{n - 2})\right),
\end{equation*}
and, for us, this gives the approximation:
\begin{equation*}
  u_t \approx \frac{u^{n + 1} - u^n}{\Delta t} = \left(\frac{3}{2} f(u^{n}) - \frac{1}{2} f(u^{n - 1})\right).
\end{equation*}
Notice the use of a forward-time difference in the derivative.

#### The implicit scheme
For the function $g$, remember that we want to use an explicit method of solution. Like the example above, for $f$, we will first consider solving:
\begin{equation*}
  u_t = g(u).
\end{equation*}
Whilst not completely explicit, the [Crank-Nicholson](https://en.wikipedia.org/wiki/Crank%E2%80%93Nicolson_method) scheme is popular and very well studied (at the time of writing Crank-Nicholson returns over $28,000$ results on Google Sholar). We evaluate $f$ at (discrete) time points $n$ and $n + 1$, and average over these values. This gives the discretisation:
\begin{equation*}
  u_t \approx \frac{u^{n + 1} - u^n}{\Delta t} = \frac{1}{2}\left(g(u^{n + 1}) - g(u^n)\right),
\end{equation*}
where we have again made use of a forward-time finite difference estimate of the time derivative.